# Ejercicio Formativo 2 Capítulo 7

## Importando librerías

In [40]:
import sqlite3

## Base de datos de fútbol europeo

### Revisando la base de datos

In [41]:
connection = sqlite3.connect('football_results.db')
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tablas:\n")
for table in cursor.fetchall():
    print(table[0])
    cursor.execute(f'PRAGMA table_info([{table[0]}])')
    print(cursor.fetchall())
    print()
connection.close()

Tablas:

Equipos
[(0, 'eid', 'INTEGER', 0, None, 1), (1, 'nombre', 'TEXT', 0, None, 0)]

Temporadas
[(0, 'tid', 'INTEGER', 0, None, 1), (1, 'ano', 'INTEGER', 0, None, 0)]

Ligas
[(0, 'lid', 'INTEGER', 0, None, 1), (1, 'nombre', 'TEXT', 0, None, 0)]

Partidos
[(0, 'pid', 'INTEGER', 0, None, 1), (1, 'liga', 'INTEGER', 0, None, 0), (2, 'temporada', 'INTEGER', 0, None, 0), (3, 'fecha', 'DATE', 0, None, 0), (4, 'equipo', 'INTEGER', 0, None, 0), (5, 'localidad', 'TEXT', 0, None, 0), (6, 'resultado', 'TEXT', 0, None, 0), (7, 'puntos', 'INTEGER', 0, None, 0), (8, 'goles', 'INTEGER', 0, None, 0), (9, 'goles_perdidos', 'INTEGER', 0, None, 0), (10, 'pases_profundos', 'INTEGER', 0, None, 0), (11, 'pases_profundos_permitidos', 'INTEGER', 0, None, 0), (12, 'ppda', 'REAL', 0, None, 0), (13, 'oppda', 'REAL', 0, None, 0)]



In [42]:
connection = sqlite3.connect('football_results.db')
cursor = connection.cursor()

In [43]:
sql = "SELECT * FROM Ligas"
cursor.execute(sql)
for r in cursor:
    print(r)

(1, 'Bundesliga')
(2, 'EPL')
(3, 'La_liga')
(4, 'Ligue_1')
(5, 'RFPL')
(6, 'Serie_A')


In [44]:
sql = "SELECT * FROM Temporadas"
cursor.execute(sql)
for r in cursor:
    print(r)

(1, 2014)
(2, 2015)
(3, 2016)
(4, 2017)
(5, 2018)
(6, 2019)


### Consultas

#### Consulta 1: Encuentre para cada liga, la cantidad de equipos distintos que han participado en ella.

In [45]:
connection = sqlite3.connect("football_results.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT L.nombre, COUNT(DISTINCT P.equipo) \
     FROM Ligas L, Partidos P WHERE L.lid = P.liga \
     GROUP BY P.liga"
)
for r in cursor:
    print(r)
connection.close()

('Bundesliga', 24)
('EPL', 30)
('La_liga', 30)
('Ligue_1', 29)
('RFPL', 25)
('Serie_A', 30)


#### Explicación

Para el desarrollo de esta consulta lo que se realizo fue una agrupación utilizando GROUP BY para la liga y luego se contó la cantidad de equipos distintos que han participado en ella, para esto se utiliza COUNT y DISTINCT para contar los equipos distintos.

#### Consulta 2: Encuentre el nombre del equipo que tiene mejor diferencia entre goles anotados y goles perdidos a lo largo de los años.

In [46]:
connection = sqlite3.connect("football_results.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT E.nombre, SUM(P.goles) - SUM(P.goles_perdidos) as diferencia\
     FROM Equipos E, Partidos P WHERE E.eid = P.equipo GROUP BY P.equipo \
         ORDER BY diferencia DESC LIMIT 1"
)
for r in cursor:
    print(r)
connection.close()

('Barcelona', 423)


#### Explicación

Para el desarrollo de esta consulta lo que se realizo fue una agrupación utilizando GROUP BY para el equipo, de esta manera se obtiene de forma agregada la información de los equipos, por ello al hacer SUM(P.goles) lo que se obtiene es la suma de los goles anotados y lo mismo para los goles perdidos, por ende al realizar la diferencia entre estos valores se obtiene la diferencia entre goles anotados y goles perdidos a lo largo de los años, luego se ordena de forma descendente y se limita a 1 para obtener el equipo con la mejor diferencia.

#### Consulta 3: Para cada liga, encuentre la temporada donde hubo mayor diferencia de puntaje entre el primer y último lugar, e indique esta diferencia.

In [47]:
connection = sqlite3.connect("football_results.db")
cursor = connection.cursor()
cursor.execute(
    "SELECT L.nombre, DPUL.temporada, MAX(DPUL.diferencia) \
     FROM Ligas L, (\
     SELECT PETL.liga, PETL.temporada, MAX(PETL.puntos_totales) - MIN(PETL.puntos_totales) AS diferencia \
     FROM (SELECT P.liga, P.temporada, P.equipo, SUM(P.puntos) AS puntos_totales \
     FROM Partidos P GROUP BY P.liga, P.temporada, P.equipo) AS PETL GROUP BY PETL.liga, PETL.temporada) \
     AS DPUL WHERE L.lid = DPUL.liga GROUP BY DPUL.liga"
)
for r in cursor:
    print(r)
connection.close()

('Bundesliga', 2, 63)
('EPL', 5, 82)
('La_liga', 1, 74)
('Ligue_1', 2, 78)
('RFPL', 3, 55)
('Serie_A', 3, 76)


#### Explicación

Antes de empezar con la explicación, se debe tener en cuenta que las subconsultas no se utilizan solo para comparar valores, sino que también se pueden utilizar para obtener una serie de datos que se necesitan para realizar una consulta más compleja, algo que se puede ver en esta consulta. Para lograr esto lo único que se hace es insertar la subconsulta dentro del FROM de la consulta que la contiene como si fuera una tabla más, de esta manera se puede utilizar la información que se obtiene de la subconsulta para realizar la consulta principal, además es recomendable asignarle un alias a la subconsulta para poder referenciarla de forma más sencilla.

Para el desarrollo de esta consulta se realizaron varias subconsultas para explicarlas de mejor manera veamos que hace cada una de estas por separado, desde la más interna a la más externa:

```sql
SELECT P.liga, P.temporada, P.equipo, SUM(P.puntos) AS puntos_totales \
     FROM Partidos P GROUP BY P.liga, P.temporada, P.equipo
```

En esta subconsulta lo que se realiza es una agrupación por liga, temporada y equipo, de esta manera se obtiene la suma de los puntos obtenidos por cada equipo en cada temporada, esta subconsulta se le asigna el alias de PETL en la subconsulta que la contiene.

Otra subconsulta, que engloba la anterior es la siguiente:

```sql
SELECT PETL.liga, PETL.temporada, MAX(PETL.puntos_totales) - MIN(PETL.puntos_totales) AS diferencia \
     FROM (SELECT P.liga, P.temporada, P.equipo, SUM(P.puntos) AS puntos_totales \
     FROM Partidos P GROUP BY P.liga, P.temporada, P.equipo) AS PETL GROUP BY PETL.liga, PETL.temporada
```

En esta subconsulta lo que se realiza es una agrupación por liga y temporada, de esta manera se obtiene la diferencia entre el primer y el último lugar, ya que el primer lugar sería el que tuvo la mayor cantidad de puntos y el último lugar el que tuvo la menor cantidad de puntos, esta subconsulta se le asigna el alias de DPUL en la subconsulta que la contiene que vendría siendo la consulta principal.

Finalmente la consulta principal es la siguiente:

```sql
SELECT L.nombre, DPUL.temporada, MAX(DPUL.diferencia) \
FROM Ligas L, (\
SELECT PETL.liga, PETL.temporada, MAX(PETL.puntos_totales) - MIN(PETL.puntos_totales) AS diferencia \
FROM (SELECT P.liga, P.temporada, P.equipo, SUM(P.puntos) AS puntos_totales \
FROM Partidos P GROUP BY P.liga, P.temporada, P.equipo) AS PETL GROUP BY PETL.liga, PETL.temporada) \
AS DPUL WHERE L.lid = DPUL.liga GROUP BY DPUL.liga
```

Lo que hace esta consulta es un JOIN entre la tabla de Ligas y la subconsulta DPUL, de esta manera se obtiene el nombre de la liga, la temporada y la diferencia entre el primer y último lugar, luego se agrupa por liga para obtener la temporada donde hubo mayor diferencia de puntaje entre el primer y último lugar.

#### Versión con WITH

In [48]:
connection = sqlite3.connect("football_results.db")
cursor = connection.cursor()
cursor.execute(
    "WITH PuntosEquipoTemporadaLiga AS \
     (SELECT P.liga as liga, P.temporada as temporada, P.equipo as equipo, SUM(P.puntos) as puntos_totales\
      FROM Partidos P \
      GROUP BY P.liga, P.temporada, P.equipo), \
      DiferenciaPrimerUltimoLugar AS \
      (SELECT PETL.liga as liga, PETL.temporada as temporada, MAX(PETL.puntos_totales) - MIN(PETL.puntos_totales) as diferencia \
      FROM PuntosEquipoTemporadaLiga PETL \
      GROUP BY PETL.liga, PETL.temporada)\
      SELECT L.nombre, DPUL.temporada, MAX(DPUL.diferencia) \
      FROM Ligas L, DiferenciaPrimerUltimoLugar DPUL WHERE L.lid = DPUL.liga\
      GROUP BY DPUL.liga"
)
for r in cursor:
    print(r)
connection.close()

('Bundesliga', 2, 63)
('EPL', 5, 82)
('La_liga', 1, 74)
('Ligue_1', 2, 78)
('RFPL', 3, 55)
('Serie_A', 3, 76)


Se muestra una versión de la consulta anterior utilizando WITH  a fin de mostrar una versión más ordenada, las subconsultas son las mismas que se explicaron anteriormente. Lo más destacable en esta ocasión es que al igual que para la versión de la consulta sin WITH, una subconsulta hecha con WITH puede utilizar otra subconsulta hecha con WITH.

## Base de datos de basketball estadounidense

Esta parte del ejercicio reutiliza la información vista para el Laboratorio 2.

### Revisando la base de datos

In [49]:
connection = sqlite3.connect('nba.sqlite')
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tablas:\n")
for table in cursor.fetchall():
    print(table[0])
    cursor.execute(f'PRAGMA table_info([{table[0]}])')
    print(cursor.fetchall())
    print()
connection.close()

Tablas:

Salaries
[(0, 'Player', 'TEXT', 0, None, 0), (1, 'Team', 'TEXT', 0, None, 0), (2, 'Salary', 'REAL', 0, None, 0)]

Stats
[(0, 'Year', 'REAL', 0, None, 0), (1, 'Player', 'TEXT', 0, None, 0), (2, 'Position', 'TEXT', 0, None, 0), (3, 'Age', 'REAL', 0, None, 0), (4, 'Team', 'TEXT', 0, None, 0), (5, 'Games', 'REAL', 0, None, 0), (6, 'GamesStarted', 'INTEGER', 0, None, 0), (7, 'MinutesPlayed', 'REAL', 0, None, 0), (8, '3Points', 'INTEGER', 0, None, 0), (9, '3PAttempts', 'INTEGER', 0, None, 0), (10, '2Points', 'REAL', 0, None, 0), (11, '2PAttempts', 'REAL', 0, None, 0), (12, 'FreeThrows', 'REAL', 0, None, 0), (13, 'FTAttempts', 'REAL', 0, None, 0), (14, 'Fouls', 'REAL', 0, None, 0), (15, 'Points', 'REAL', 0, None, 0)]



### Consultas

#### Consulta 1: Complete los valores faltantes para la columna 3Points, cuando sea posible inferirlos a partir del resto de las columnas.

In [50]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    'SELECT "3Points", "2Points", FreeThrows, Points FROM Stats S WHERE "3Points" IS NULL'
)
for r in cursor:
    print(r)
connection.close()

Esta consulta es solo para revisar los valores faltantes en la columna 3Points. Aquí pueden ver que no muestra ningún resultado pero esto es debido a que al ejecutar el código por segunda vez en adelante, los valores faltantes ya han sido completados.

In [51]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    'UPDATE Stats SET "3Points" = (Points - ((2*"2Points") + FreeThrows))/3 WHERE "3Points" IS NULL AND "2Points" IS NOT NULL \
        AND FreeThrows IS NOT NULL AND Points IS NOT NULL'
)
connection.commit()
connection.close()

#### Explicación

Para completar los valores faltantes en la columna 3Points, se tuvo en cuenta que el total de puntos, indicado por la columna Points, sería equivalente a los puntos logrados por tiros de 3 puntos, tiros de 2 puntos y tiros libres. Por lo tanto, se calculó la cantidad de puntos que se obtendrían por tiros de 2 puntos y tiros libres, y se restó este valor al total de puntos para obtener los puntos obtenidos por tiros de 3 puntos y se dividió por 3 para obtener la cantidad de tiros de 3 puntos realizados.

Para lograr actualizar estos datos se utilizo UPDATE y SET para actualizar los valores de la columna 3Points, además se utilizo WHERE para indicar que se actualicen solo los valores que son nulos cuando los valores de las columnas 2Points, FreeThrows y Points no son nulos. La sintaxis para utilizar UPDATE es la siguiente:

```sql
UPDATE nombre_tabla SET columna1 = valor1, columna2 = valor2, ... WHERE condicion;
```

In [52]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    'SELECT * FROM Stats S WHERE "3Points" IS NULL'
)
for r in cursor:
    print(r)
connection.close()

Esta consulta se utiliza para verificar que los datos ya fueron completados

#### Consulta 2: Encuentre los 5 equipos de la NBA para los que sus 5 jugadores más veteranos cometieron más fouls. Para cada equipo, imprima además la cantidad de fouls agregada que realizaron estos jugadores. Ordene los resultados de manera ascendente en base a los fouls.

In [53]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    "WITH RankingJugadoresVeteranosEquipo AS \
     (SELECT S.Team as Team, S.Player as Player, ROW_NUMBER() OVER(PARTITION BY S.Team ORDER BY S.Age DESC) AS \
     ranking_edad FROM Stats S), \
     FoulsTotalesJugador AS \
     (SELECT S.Team as Team, S.Player as Player, SUM(S.Fouls) as total_fouls FROM Stats S GROUP BY S.Team, S.Player), \
     Top5VeteranosFoulsTotales AS \
     (SELECT RJVE.Team as Team, RJVE.Player as Player, FTJ.total_fouls as total_fouls \
      FROM RankingJugadoresVeteranosEquipo RJVE, FoulsTotalesJugador FTJ \
      WHERE RJVE.Team = FTJ.Team AND RJVE.Player = FTJ.Player AND ranking_edad <= 5) \
      SELECT T5VFT.Team, SUM(T5VFT.total_fouls) as total_fouls_agregada FROM Top5VeteranosFoulsTotales T5VFT \
      GROUP BY T5VFT.Team ORDER BY total_fouls_agregada DESC LIMIT 5"
)
for r in cursor:
    print(r)
connection.close()

('MEM', 698.0)
('BOS', 686.0)
('POR', 661.0)
('NYK', 588.0)
('TOR', 576.0)


#### Explicación

Antes de empezar se explicará la función ROW_NUMBER() que se utilizó en esta consulta, esta función asigna un número a cada fila de un conjunto de resultados, comenzando en 1 para la primera fila de cada partición. Se puede utilizar para clasificar un conjunto de resultados por una columna o para asignar un número de fila a cada fila. La sintaxis de esta función es la siguiente:

```sql
ROW_NUMBER() OVER ( [PARTITION BY columna1, columna2, ...] ORDER BY columna1, columna2, ... ) AS alias
```

Para el desarrollo de esta consulta se utilizaron varias consultas hechas con WITH, la primera de ellas es la siguiente:

```sql
RankingJugadoresVeteranosEquipo AS \
(SELECT S.Team as Team, S.Player as Player, ROW_NUMBER() OVER(PARTITION BY S.Team ORDER BY S.Age DESC) AS \
ranking_edad FROM Stats S)
```

En esta consulta lo que se realiza es una partición por equipo y se ordena de forma descendente por la edad de los jugadores, de esta manera se obtiene el ranking de los jugadores más veteranos de cada equipo, el ranking se señala por el valor de la columna ranking_edad.

```sql
FoulsTotalesJugador AS \
(SELECT S.Team as Team, S.Player as Player, SUM(S.Fouls) as total_fouls FROM Stats S GROUP BY S.Team, S.Player)
```

En esta consulta lo que se realiza es una agrupación por equipo y jugador, de esta manera se obtiene la suma de los fouls realizados por cada jugador de cada equipo, el total de fouls se señala por el valor de la columna total_fouls.

```sql
Top5VeteranosFoulsTotales AS \
(SELECT RJVE.Team as Team, RJVE.Player as Player, FTJ.total_fouls as total_fouls \
FROM RankingJugadoresVeteranosEquipo RJVE, FoulsTotalesJugador FTJ \
WHERE RJVE.Team = FTJ.Team AND RJVE.Player = FTJ.Player AND ranking_edad <= 5)
```

En esta consulta lo que se realiza es un JOIN entre las consultas RankingJugadoresVeteranosEquipo y FoulsTotalesJugador, de esta manera se obtiene una nueva tabla con los jugadores con su respectivo ranking y total de fouls, además se filtra para obtener solo los 5 jugadores más veteranos de cada equipo.

Finalmente la consulta principal es la siguiente:

```sql
SELECT T5VFT.Team, SUM(T5VFT.total_fouls) as total_fouls_agregada FROM Top5VeteranosFoulsTotales T5VFT \
GROUP BY T5VFT.Team ORDER BY total_fouls_agregada DESC LIMIT 5
```

Lo que hace esta consulta es un agrupar por equipo a la tabla Top5VeteranosFoulsTotales, de esta manera se obtiene la suma de los fouls realizados por los 5 jugadores más veteranos de cada equipo, luego se ordena de forma descendente y se limita a 5 para obtener los 5 equipos de la NBA para los que sus 5 jugadores más veteranos cometieron más fouls.

#### Consulta 3: Encuentre los 10 jugadores más costosos de la NBA de acuerdo a la cantidad de dinero que les pagaron por cada punto que convirtieron. Para cada jugador, imprima además su sueldo total en la temporada. Ordene los resultados de manera descendente, de acuerdo al sueldo total.

In [54]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    "SELECT Player, salario_total \
     FROM (SELECT S.Player as Player, SUM(A.Salary) as salario_total, SUM(A.Salary)/SUM(S.Points) as salario_punto \
     FROM Stats S, Salaries A WHERE S.Player = A.Player AND S.Team = A.Team\
     GROUP BY S.Player ORDER BY salario_punto DESC LIMIT 10) ORDER BY salario_total DESC"
)
for r in cursor:
    print(r)
connection.close()

('Chandler Parsons', 23112004.0)
('Joakim Noah', 17765000.0)
('Ian Mahinmi', 16661641.0)
('Jerryd Bayless', 9000000.0)
('Lance Stephenson', 4180000.0)
('Udonis Haslem', 2328652.0)
('Chris McCullough', 1471382.0)
('Josh Huestis', 1471382.0)
('Chinanu Onuaku', 1312611.0)
('Michael Gbinije', 500000.0)


#### Explicación

Nuevamente se explicará lo que hace cada subconsulta antes de explicar la consulta principal, aunque en esta ocasión solo hay una subconsulta.

```sql
SELECT S.Player as Player, SUM(A.Salary) as salario_total, SUM(A.Salary)/SUM(S.Points) as salario_punto \
FROM Stats S, Salaries A WHERE S.Player = A.Player AND S.Team = A.Team\
GROUP BY S.Player ORDER BY salario_punto DESC LIMIT 10
```

Lo que hace esta consulta es un JOIN entre las tablas Stats y Salaries, se agrupa por jugador y se obtiene la suma del salario total y el salario por punto, luego se ordena de forma descendente y se limita a 10 para obtener los 10 jugadores más costosos de la NBA de acuerdo a la cantidad de dinero que les pagaron por cada punto que convirtieron.

Luego en la consulta principal:

```sql
SELECT Player, salario_total \
FROM (SELECT S.Player as Player, SUM(A.Salary) as salario_total, SUM(A.Salary)/SUM(S.Points) as salario_punto \
FROM Stats S, Salaries A WHERE S.Player = A.Player AND S.Team = A.Team\
GROUP BY S.Player ORDER BY salario_punto DESC LIMIT 10) ORDER BY salario_total DESC
```
Lo que hace esta consulta es un SELECT de la subconsulta anterior, de esta manera se obtiene el nombre del jugador y su salario total en la temporada, luego se ordena de forma descendente de acuerdo al salario total.

#### Consulta 4: Para los jugadores que fueron parte de 2 o más equipos distintos durante la temporada, encuentre aquel que tuvo la mejora más significativa en cuanto a puntos convertidos por partido.

In [55]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    "SELECT PPJ1.Player, MAX(PPJ1.avg_points - PPJ2.avg_points) AS max_mejora \
     FROM \
     (SELECT S.Team as Team, S.Player as Player, AVG(S.Points) as avg_points \
     FROM Stats S WHERE S.Player IN (SELECT S.Player FROM Stats S GROUP BY S.Player \
     HAVING COUNT(DISTINCT S.Team) >= 2) GROUP BY S.Team, S.Player \
     ) AS PPJ1, \
    (SELECT S.Team as Team, S.Player as Player, AVG(S.Points) as avg_points \
     FROM Stats S WHERE S.Player IN (SELECT S.Player FROM Stats S GROUP BY S.Player \
     HAVING COUNT(DISTINCT S.Team) >= 2) GROUP BY S.Team, S.Player \
     ) AS PPJ2 \
     WHERE PPJ1.Player = PPJ2.Player AND PPJ1.Team <> PPJ2.Team \
     GROUP BY PPJ1.Player \
     ORDER BY max_mejora DESC \
     LIMIT 1"
)
for r in cursor:
    print(r)
connection.close()


('DeMarcus Cousins', 1528.0)


#### Explicación

Para esta consulta también se utilizan varias subconsultas, se procederá a analizar cada una de ellas por separado:
    
```sql
SELECT S.Player FROM Stats S GROUP BY S.Player \
     HAVING COUNT(DISTINCT S.Team) >= 2
```

Esta subconsulta lo que hace es una agrupación por jugador, y utilizando HAVING se filtra para obtener solo los jugadores que fueron parte de 2 o más equipos distintos durante la temporada.

```sql
SELECT S.Team as Team, S.Player as Player, AVG(S.Points) as avg_points \
     FROM Stats S WHERE S.Player IN (SELECT S.Player FROM Stats S GROUP BY S.Player \
     HAVING COUNT(DISTINCT S.Team) >= 2) GROUP BY S.Team, S.Player
```

Esta subconsulta lo que hace es agrupar por equipo y por jugador, de esta manera se obtiene el promedio de puntos convertidos por partido de cada jugador en cada equipo, además se filtra para obtener solo los jugadores que fueron parte de 2 o más equipos distintos durante la temporada.

Luego en la consulta principal se hace lo siguiente:

```sql
SELECT PPJ1.Player, MAX(PPJ1.avg_points - PPJ2.avg_points) AS max_mejora \
FROM \
(SELECT S.Team as Team, S.Player as Player, AVG(S.Points) as avg_points \
FROM Stats S WHERE S.Player IN (SELECT S.Player FROM Stats S GROUP BY S.Player \
HAVING COUNT(DISTINCT S.Team) >= 2) GROUP BY S.Team, S.Player \
) AS PPJ1, \
(SELECT S.Team as Team, S.Player as Player, AVG(S.Points) as avg_points \
FROM Stats S WHERE S.Player IN (SELECT S.Player FROM Stats S GROUP BY S.Player \
HAVING COUNT(DISTINCT S.Team) >= 2) GROUP BY S.Team, S.Player \
) AS PPJ2 \
WHERE PPJ1.Player = PPJ2.Player AND PPJ1.Team <> PPJ2.Team \
GROUP BY PPJ1.Player \
ORDER BY max_mejora DESC \
LIMIT 1
```

Como pueden ver se hace uso un JOIN de la misma tabla(subconsulta), de esta ,manera se obtiene las posibles comparaciones entre el cambio de puntos por partido de un jugador al pasar de un equipo a otro, luego se agrupa por jugador y se calcula la mejora más significativa en cuanto a puntos convertidos por partido al cambiarse de equipo de entre todas las que tuvo el jugador, finalmente se ordena de forma descendente respecto a esta máxima mejora y se limita a 1 para obtener el jugador que tuvo la mejora más significativa en cuanto a puntos convertidos por partido.


#### Versión con WITH

In [56]:
connection = sqlite3.connect("nba.sqlite")
cursor = connection.cursor()
cursor.execute(
    "WITH JugadorParticipoVariosEquipos AS \
     (SELECT S.Player as Player FROM Stats S GROUP BY S.Player HAVING COUNT (DISTINCT S.Team) >= 2), \
      PromedioPuntosJugador AS \
      (SELECT S.Team as Team, S.Player as Player, AVG(S.Points) as avg_points FROM Stats S WHERE S.Player IN \
      (SELECT JPVE.Player FROM JugadorParticipoVariosEquipos JPVE) \
       GROUP BY S.Team, S.Player), \
       JugadoresMejorasEntreEquipos AS ( \
       SELECT PPJ1.Player, MAX(PPJ1.avg_points - PPJ2.avg_points) AS max_mejora \
       FROM PromedioPuntosJugador PPJ1, PromedioPuntosJugador PPJ2 \
       WHERE PPJ1.Player = PPJ2.Player AND PPJ1.Team <> PPJ2.Team \
       GROUP BY PPJ1.Player) \
        SELECT * FROM JugadoresMejorasEntreEquipos ORDER BY max_mejora DESC LIMIT 1"
)
for r in cursor:
    print(r)
connection.close()

('DeMarcus Cousins', 1528.0)


Se deja una versión utilizando WITH para mostrar una versión más ordenada de la consulta anterior, las subconsultas son las mismas que se explicaron anteriormente.